In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=10
✅ 시뮬레이션 초기화 완료: S=10, Randomness='high', M1=758.40, M2=1855.82


/Users/jangseohyun/Documents/workspace/symply/DER/functions.py:95: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")


In [3]:
only = gp.Model("only")
# only.setParam("MIPGap", 1e-7)

x = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z")
zc = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
zeta = only.addVars(I, T, S, vtype=GRB.BINARY, name="zeta")
delta = only.addVars(I, T, S, vtype=GRB.BINARY, name="delta")

only.update()

obj = gp.quicksum(P_DA[t] * x[i, t] for i, t in product(range(I), range(T))) \
    + gp.quicksum(1 / S * (P_RT[t, s] * yp[i, t, s] - P_PN[t] * ym[i, t, s]) for i, t, s in product(range(I), range(T), range(S)))
only.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    only.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s] + zc[i, t, s] - zd[i, t, s])
    only.addConstr(R[i, t, s] + zd[i, t, s] >= yp[i, t, s] + zc[i, t, s])
    only.addConstr(ym[i, t, s] <= M1 * delta[i, t, s])
    only.addConstr(zc[i, t, s] <= M1 * (1 - delta[i, t, s]))
    only.addConstr(zc[i, t, s] <= M1 * zeta[i, t, s])
    only.addConstr(zd[i, t, s] <= M1 * (1 - zeta[i, t, s]))
    only.addConstr(z[i, t, s] <= K[i])
    only.addConstr(z[i, t, s] >= 0)
for i, s in product(range(I), range(S)):
    only.addConstr(z[i, 0, s] == K0[i] + zc[i, 0, s] - zd[i, 0, s])
for i, t, s in product(range(I), range(1, T), range(S)):
    only.addConstr(z[i, t, s] == z[i, t - 1, s] + zc[i, t, s] - zd[i, t, s])

only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {only.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10800 rows, 8520 columns and 26350 nonzeros
Model fingerprint: 0x117b769b
Variable types: 6120 continuous, 2400 integer (2400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 8e+02]
Presolve removed 9480 rows and 7116 columns
Presolve time: 0.08s
Presolved: 1320 rows, 1404 columns, 3880 nonzeros
Variable types: 1064 continuous, 340 integer (340 binary)
Found heuristic solution: objective 1675095.9983

Root relaxation: objective 1.709055e+06, 792 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1709055.42    0    2 16750

In [4]:
da_profit = sum(P_DA[t] * x[i, t].X for i in range(I) for t in range(T))
rt_profit = sum(P_RT[t, s] * yp[i, t, s].X / S for i in range(I) for t in range(T) for s in range(S))
pn_cost   = sum(P_PN[t] * ym[i, t, s].X / S for i in range(I) for t in range(T) for s in range(S))
total_profit = da_profit + rt_profit - pn_cost

print("[DISAGGREGATION MODEL]")
print(f"DA Profit      = {da_profit:.2f}")
print(f"RT Profit      = {rt_profit:.2f}")
print(f"Penalty Cost   = {pn_cost:.2f}")
print(f"Total Profit   = {total_profit:.2f}")

[DISAGGREGATION MODEL]
DA Profit      = 200157.14
RT Profit      = 1515686.89
Penalty Cost   = 7078.31
Total Profit   = 1708765.71


In [5]:
print("\n[ONLY MODEL] Day-Ahead Commitment (sum over DERs):")
total_only_commit = 0
for t in range(T):
    commit_t = sum(x[i, t].X for i in range(I))
    total_only_commit += commit_t
    print(f"Time {t}: {commit_t:.2f}")
print(f"TOTAL: {total_only_commit:.2f}")


[ONLY MODEL] Day-Ahead Commitment (sum over DERs):
Time 0: 0.00
Time 1: 0.00
Time 2: 0.00
Time 3: 0.00
Time 4: 0.00
Time 5: 0.00
Time 6: 0.00
Time 7: 0.00
Time 8: 0.00
Time 9: 48.21
Time 10: 226.18
Time 11: 0.00
Time 12: 0.00
Time 13: 0.00
Time 14: 383.37
Time 15: 0.00
Time 16: 239.75
Time 17: 0.00
Time 18: 0.00
Time 19: 342.65
Time 20: 109.07
Time 21: 32.50
Time 22: 0.00
Time 23: 0.00
TOTAL: 1381.73
